In [ ]:
import numpy as np
import pyvista as pv
import torch
import matplotlib.pyplot as plt
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [ ]:
def slide2D_fractionZ(film, start, end, direction, fraction, value):
    if fraction == '+':
        if direction == 'y':
            slit = np.linspace(0, np.abs(end[2] - start[2]), np.abs(end[1] - start[1]))
            fraction = np.abs(int(slit[0]-slit[1]))
            print('y', slit)
            print('fraction', fraction)
            for i in range(np.abs(end[1] - start[1])):
                if end[1] > start[1]:
                    film[start[0]:end[0], start[1] + i, start[2]:start[2] + int(slit[i])] = value
                    for j in range(fraction):
                        film[start[0]:end[0], start[1] + i,start[2]+int(slit[i])+j] = 1/(fraction+1)*(fraction-j)
                elif end[1] < start[1]:
                    film[start[0]:end[0], start[1] - i, start[2]:start[2] + int(slit[i])] = value
                    for j in range(fraction):
                        film[start[0]:end[0], start[1] - i,start[2]+int(slit[i])+j] = 1/(fraction+1)*(fraction-j)
        elif direction == 'x':
            slit = np.linspace(0, np.abs(end[2] - start[2]), np.abs(end[0] - start[0]))
            fraction = np.abs(int(slit[0]-slit[1]))
            print('x', slit)
            print('fraction', fraction)
            for i in range(np.abs(end[2] - start[2])):
                if end[0] > start[0]:
                    film[start[0] + i, start[1]:end[1], start[2]:start[2] + int(slit[i])] = value
                    for j in range(fraction):
                        film[start[0] + i, start[1]:end[1], start[2]+int(slit[i])+j] = 1/(fraction+1)*(fraction-j)
                elif end[0] < start[0]:
                    film[start[0] - i, start[1]:end[1], start[2]:start[2] + int(slit[i])] = value
                    for j in range(fraction):
                        film[start[0] - i, start[1]:end[1], start[2]+int(slit[i])+j] = 1/(fraction+1)*(fraction-j)
    elif fraction == '-':
        if direction == 'y':
            slit = np.linspace(0, np.abs(end[2] - start[2]), np.abs(end[1] - start[1]))
            fraction = np.abs(int(slit[0]-slit[1]))
            print('y', slit)
            print('fraction', fraction)
            for i in range(np.abs(end[1] - start[1])):
                if end[1] > start[1]:
                    film[start[0]:end[0], start[1] + i, start[2] - int(slit[i]):start[2]+1] = value
                    for j in range(fraction):
                        film[start[0]:end[0], start[1] + i,start[2]-int(slit[i])-j] = 1/(fraction+1)*(fraction-j)
                elif end[1] < start[1]:
                    film[start[0]:end[0], start[1] - i, start[2] - int(slit[i]):start[2]+1] = value
                    for j in range(fraction):
                        film[start[0]:end[0], start[1] - i,start[2]-int(slit[i])-j] = 1/(fraction+1)*(fraction-j)
        elif direction == 'x':
            slit = np.linspace(0, np.abs(end[2] - start[2]), np.abs(end[0] - start[0]))
            fraction = np.abs(int(slit[0]-slit[1]))
            print('x', slit)
            print('fraction', fraction)
            for i in range(np.abs(end[2] - start[2])):
                if end[0] > start[0]:
                    film[start[0] + i, start[1]:end[1], start[2] - int(slit[i]):start[2]+1] = value
                    for j in range(fraction):
                        film[start[0] + i, start[1]:end[1], start[2] - int(slit[i]):start[2]] = 1/(fraction+1)*(fraction-j)
                elif end[0] < start[0]:
                    film[start[0] - i, start[1]:end[1], start[2] - int(slit[i]):start[2]+1] = value
                    for j in range(fraction):
                        film[start[0] - i, start[1]:end[1], start[2] - int(slit[i]):start[2]] = 1/(fraction+1)*(fraction-j)
    return film

In [ ]:
film = np.zeros((70, 200, 150))

bottom = 10
# film[:, :, 0:bottom] = 10 # bottom

height = 80
left_side = 71
right_side = 71

slit = 8
film[:, left_side+slit:200-right_side-slit, 0:height] = 10

left_side_gap = 19
right_side_gap = 181
film[:, :left_side_gap, 0:height] = 10
film[:, right_side_gap:, 0:height] = 10

film = slide2D_fractionZ(film=film, start=[0, left_side, bottom], end=[70, left_side+slit, height], direction='y', fraction='+', value=10)
film = slide2D_fractionZ(film=film, start=[0, 200-right_side-1, bottom], end=[70, 200-right_side-slit-1, height], direction='y', fraction='+', value=10)
film = slide2D_fractionZ(film=film, start=[0, left_side_gap+slit-1, bottom], end=[70, left_side_gap-1, height], direction='y', fraction='+', value=10)
film = slide2D_fractionZ(film=film, start=[0, right_side_gap-slit, bottom], end=[70, right_side_gap, height], direction='y', fraction='+', value=10)

# film[:, 80:121, 0:31] = 10

film[:, :, 0:bottom] = 10 # bottom
film[:, :, height:] = 0 # bottom

In [ ]:
geom = pv.Box()

substrute = torch.Tensor(film==10).to_sparse()
substrute = substrute.indices().numpy().T

cyan = torch.Tensor(np.logical_and(film[:, :, :]!=10, film[:, :, :]!=0)).to_sparse()
cyan = cyan.indices().numpy().T

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

cyanmesh = pv.PolyData(cyan)
cyanmesh["radius"] = np.ones(cyan.shape[0])*0.5
# Progress bar is a new feature on master branch
cyanglyphed = cyanmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)
p = pv.Plotter()
p.add_mesh(subglyphed, color='gray')
# p.add_mesh(cyanglyphed, color='cyan')
p.enable_eye_dome_lighting()
p.show()


In [ ]:
etchfilm = np.zeros((70, 200, 150, 2))
etchfilm[:, :, :, 0] = film
# etchfilm[:, :, :, 1] = film

center = 100


In [ ]:
etchfilm = np.load('./sputter_etching_paper_depo_1113.npy')
center = 100

In [ ]:
indice = etchfilm[:, :, :, 0] < 1
etchfilm[indice, 0] = 0

In [ ]:
import Multi_Species_bosch_etching_countByReaction_mirror_func_redepo

In [ ]:
from scipy import interpolate

In [ ]:
yield_hist = np.array([[1.0, 1.01, 1.05,  1.2,  1.4,  1.5, 1.07, 0.65, 0.28, 0.08,  0, \
                        0.08, 0.28,0.65,  1.07, 1.5, 1.4, 1.2, 1.05, 1.01, 1.0 ], \
                        [  0,  5,   10,   20,   30,   40,   50,   60,   70,   80, 90, \
                          100, 110, 120, 130, 140, 150, 160, 170, 175, 180]])
yield_hist[1] *= np.pi/180
yield_func = interpolate.interp1d(yield_hist[1], yield_hist[0], kind='quadratic')

xnew = np.arange(0, np.pi/2, 0.01)
# xnew = np.array([])
ynew = yield_func(xnew)   # use interpolation function returned by `interp1d`
plt.plot(yield_hist[1, :11], yield_hist[0, :11], 'o', xnew, ynew, '-')
plt.show()

In [ ]:
def Rn_coeffcient(c1, c2, c3, c4, alpha):
    return c1 + c2*np.tanh(c3*alpha - c4)

rn_angle = np.arange(0, np.pi/2, 0.1)
# xnew = np.array([])
rn_prob = [Rn_coeffcient(0.9423, 0.9434, 2.342, 3.026, i) for i in rn_angle]
rn_prob /= rn_prob[-1]

yield_hist[1] *= np.pi/180
xnew = np.arange(0, np.pi/2, 0.01)

react_rate = np.interp(xnew, rn_angle, rn_prob)

# yield_func = np.interp(xnew, yield_hist[1], yield_hist[0])

plt.plot(rn_angle, rn_prob, 'o', xnew, react_rate, '-')
plt.show()

In [ ]:
gamma0 = 1
gammaMax = 0.5
thetaMax = np.pi/5
f = -np.log(gammaMax/gamma0)/(np.log(np.cos(gammaMax)) + 1 - np.cos(thetaMax))
s = f*np.cos(thetaMax)

theta = np.arange(0, np.pi/2, 0.01)
sputterYield = gamma0*np.cos(theta)**(-f)*np.exp(-s*(1/np.cos(theta) - 1))

yield_hist = np.zeros((2, theta.shape[0]))

yield_hist[0, :] = sputterYield
yield_hist[1, :] = theta

plt.plot(yield_hist[1, :], yield_hist[0, :], '-')
plt.show()

In [ ]:
logname = 'Multi_species_benchmark_1113_redepo'
inputMethod='bunch'
etchingPoint = np.array([center, center, 125])
depoPoint = np.array([center, center, 125])
density = 10
center_with_direction=np.array([[int(etchfilm.shape[0]/2),int(etchfilm.shape[1]/2),150]])
range3D=np.array([[0, etchfilm.shape[0], 0, etchfilm.shape[1], 0, etchfilm.shape[2]]])
InOrOut=[1]
# yield_hist=np.array([None])
yield_hist = yield_hist
maskTop=40, 
maskBottom=98, 
maskStep=10, 
maskCenter=[int(etchfilm.shape[0]/2), int(etchfilm.shape[1]/2)]
backup=False
mirrorGap=5
reaction_type=False
param = [1.6, -0.7]
n=1
celllength=1e-5
kdtreeN=5
filmKDTree=np.array([[2, 0, 1], [3, 0, -1]]) # 1 for depo -1 for etching
# filmKDTree=np.array([[2, 1], [3, 1]])
weightDepo=0.2
weightEtching = 0.2
tstep=1e-5
substrateTop=80
posGeneratorType='top'
testEtch = Multi_Species_bosch_etching_countByReaction_mirror_func_redepo.etching(
                    inputMethod,
                    etchingPoint,depoPoint,
                    density, center_with_direction, 
                    range3D, InOrOut, yield_hist,
                    maskTop, maskBottom, maskStep, maskCenter,backup, 
                    mirrorGap,
                    reaction_type, param,n,
                    celllength, kdtreeN, filmKDTree,weightDepo,weightEtching, tstep,
                    substrateTop,posGeneratorType, logname)

In [ ]:
react_table = np.array([[[0.0, 0, 1], [0.0, 0, 1]],
                        [[0.00, -1, 0], [0.00, 0, -1]]])

react_type_table = np.array([[2, 0],
                             [3, 0]])

In [ ]:
Multi_Species_bosch_etching_countByReaction_mirror_func_redepo.react_table = react_table
Multi_Species_bosch_etching_countByReaction_mirror_func_redepo.react_type_table = react_type_table

In [ ]:
cicle = 100
celllength=1e-5
parcel = np.array([[95*celllength, 95*celllength, 159*celllength, 0, 0, 1, 95, 95, 159, 0.2, 0]])
step1 = testEtch.inputParticle(etchfilm, parcel, 'depo', 'maxwell', 0, 0, int(1e4), int(1e6), int(6e6),2, 4, 100)

In [ ]:
cicle = 100
celllength=1e-5
parcel = np.array([[95*celllength, 95*celllength, 159*celllength, 0, 0, 1, 95, 95, 159,0.2, 0]])
step1 = testEtch.inputParticle(etchfilm, parcel, 'etching', 'updown',0, 1, int(5e3), int(1e6), int(1e6),3, 10, 100)

In [ ]:
etchfilm[20, 54, :, 0]

In [ ]:
geom = pv.Box()

substrute = torch.Tensor(film[:, :, :]==10).to_sparse()
substrute = substrute.indices().numpy().T

cyan = torch.Tensor(np.logical_and(np.logical_and(etchfilm[:, :, :,0]>=9, etchfilm[:, :, :,0]<10), film[:, :, :]!=10)).to_sparse()
# cyan = torch.Tensor(etchfilm[:, :, :,0]==10).to_sparse()
cyan = cyan.indices().numpy().T

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

cyanmesh = pv.PolyData(cyan)
cyanmesh["radius"] = np.ones(cyan.shape[0])*0.5
# Progress bar is a new feature on master branch
cyanglyphed = cyanmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)
p = pv.Plotter()
p.add_mesh(subglyphed, color='gray')
p.add_mesh(cyanglyphed, color='cyan')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
pos1e4_data = testEtch.planes
point_cloud = pv.PolyData(pos1e4_data[:, 3:6])
vectors = pos1e4_data[:, :3]

point_cloud['vectors'] = vectors
arrows = point_cloud.glyph(
    orient='vectors',
    scale=10000,
    factor=3,
)

# Display the arrowscyan


plotter = pv.Plotter()
plotter.add_mesh(point_cloud, color='cyan', point_size=5.0, render_points_as_spheres=True)
# plotter.add_mesh(sphere, show_edges=True, opacity=0.5, color="w")
plotter.add_mesh(arrows, color='lightblue')
# # plotter.add_point_labels([point_cloud.center,], ['Center',],
# #                          point_color='yellow', point_size=20)
plotter.show_grid()
plotter.show()

In [ ]:
geom = pv.Box()

substrute = torch.Tensor(film[:, :, :]==10).to_sparse()
substrute = substrute.indices().numpy().T

cyan = torch.Tensor(np.logical_and(etchfilm[:, :, :,0]>=9, film[:, :, :]!=10)).to_sparse()
# cyan = torch.Tensor(etchfilm[:, :, :,0]==10).to_sparse()
cyan = cyan.indices().numpy().T

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

cyanmesh = pv.PolyData(cyan)
cyanmesh["radius"] = np.ones(cyan.shape[0])*0.5
# Progress bar is a new feature on master branch
cyanglyphed = cyanmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)
p = pv.Plotter()
p.add_mesh(subglyphed, color='gray')
p.add_mesh(cyanglyphed, color='cyan')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
geom = pv.Box()

beforeEtch = np.load('./sputter_etching_paper_depo_1113.npy')

substrute = torch.Tensor(film[:, :, :]==10).to_sparse()
substrute = substrute.indices().numpy().T

red_indice = np.logical_and(beforeEtch[:, :, :,0]==10, etchfilm[:, :, :, 0]!=10)
# red_indice |= etchfilm[:, :, :,0] != 10

red = torch.Tensor(red_indice).to_sparse()
red = red.indices().numpy().T

blue_indice = np.logical_and(beforeEtch[:, :, :,0]==0, etchfilm[:, :, :, 0]>=9)
# red_indice |= etchfilm[:, :, :,0] != 10

blue = torch.Tensor(blue_indice).to_sparse()
blue = blue.indices().numpy().T


# cyan = torch.Tensor(cyan_indice).to_sparse()
cyan = torch.Tensor(np.logical_and(etchfilm[:, :, :,0]==10, film[:, :, :]!=10)).to_sparse()
# cyan = torch.Tensor(etchfilm[:, :, :,0]==10).to_sparse()
cyan = cyan.indices().numpy().T

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

cyanmesh = pv.PolyData(cyan)
cyanmesh["radius"] = np.ones(cyan.shape[0])*0.5

redmesh = pv.PolyData(red)
redmesh["radius"] = np.ones(red.shape[0])*0.5

bluemesh = pv.PolyData(blue)
bluemesh["radius"] = np.ones(blue.shape[0])*0.5
# Progress bar is a new feature on master branch
cyanglyphed = cyanmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)
redglyphed = redmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
blueglyphed = bluemesh.glyph(scale="radius", geom=geom) # progress_bar=True)
p = pv.Plotter()
p.add_mesh(subglyphed, color='gray', opacity=0.1)
p.add_mesh(cyanglyphed, color='cyan')
p.add_mesh(redglyphed, color='red')
# p.add_mesh(blueglyphed, color='blue')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
geom = pv.Box()

beforeEtch = np.load('./sputter_etching_paper_depo_1113.npy')

substrute = torch.Tensor(film[:, :, :]==10).to_sparse()
substrute = substrute.indices().numpy().T

red_indice = np.logical_and(beforeEtch[:, :, :,0]==10, film[:, :, :]!=10)
# red_indice |= etchfilm[:, :, :,0] != 10

red = torch.Tensor(red_indice).to_sparse()
red = red.indices().numpy().T

# cyan = torch.Tensor(cyan_indice).to_sparse()
cyan = torch.Tensor(np.logical_and(etchfilm[:, :, :,0]==10, film[:, :, :]!=10)).to_sparse()
# cyan = torch.Tensor(etchfilm[:, :, :,0]==10).to_sparse()
cyan = cyan.indices().numpy().T

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

cyanmesh = pv.PolyData(cyan)
cyanmesh["radius"] = np.ones(cyan.shape[0])*0.5

redmesh = pv.PolyData(red)
redmesh["radius"] = np.ones(red.shape[0])*0.5
# Progress bar is a new feature on master branch
cyanglyphed = cyanmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)
redglyphed = redmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
p = pv.Plotter()
p.add_mesh(subglyphed, color='gray', opacity=0.1)
# p.add_mesh(cyanglyphed, color='cyan')
p.add_mesh(redglyphed, color='cyan')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
np.save('./sputter_etching_paper_depo_1113_NoRedepo', etchfilm)